In [ ]:
!pip install -q pypdf python-dotenv transformers llama-index llama-index-postprocessor-colbert-rerank llama-index-embeddings-huggingface llama-index-embeddings-instructor sentence-transformers llama-index-vector-stores-chroma chromadb flask flask_ngrok pyngrok flask-cors

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.78-cp310-cp310-linux_x86_64.whl size=169125856 sha256=0c479fe3869f76e62d5b8d68b6850d685898a001805b7b49db122af791fdbcad
  Stored in directory: /tmp/pip-ephem-wheel-cache-7zewhgud/wheels/fd/c5/bd/3b1c20081bd71ce9d28b562573c97915c790bf1ef231879a61
Successfully built llama-cpp-python


In [ ]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,ServiceContext,Settings
API_KEY="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"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r ./drive/MyDrive/chemistri .

In [ ]:
# documents = SimpleDirectoryReader("/content/Data/").load_data()

In [ ]:
%pip install llama-index-llms-llama-cpp llama-index-tools-tavily-research

In [ ]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q8_0.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=512,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4000,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    # messages_to_prompt=messages_to_prompt,
    # completion_to_prompt=completion_to_prompt,
    verbose=True,
)
Settings.llm=llm
Settings.chunk_size=512

total size (MB): 7695.86


 44%|████▎     | 3199/7339 [06:17<08:46,  7.86it/s]

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
import chromadb
from chromadb.config import Settings

In [ ]:
# index = VectorStoreIndex.from_documents(documents)

In [ ]:
db = chromadb.PersistentClient(path="./chemistri")

# get collection
chroma_collection = db.get_or_create_collection("documents")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context,embed_model=embed_model
)

In [ ]:
# db = chromadb.PersistentClient(path="./vectors",settings=Settings(anonymized_telemetry=False))

# # get collection
# chroma_collection = db.get_or_create_collection("documents")

# # assign chroma as the vector_store to the context
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# # load your index from stored vectors
# index = VectorStoreIndex.from_vector_store(
#     vector_store, storage_context=storage_context,embed_model=embed_model
# )

In [ ]:
ret = index.as_retriever()
print([n.text[0:300] for n in ret.retrieve("what is physical chemistry?")])


In [ ]:
# from llama_index.postprocessor.colbert_rerank import ColbertRerank
# # from llama_index.core.query_engine import RetrieverQueryEngine
# colbert_reranker = ColbertRerank(
#     top_n=3,
#     model="colbert-ir/colbertv2.0",
#     tokenizer="colbert-ir/colbertv2.0",
# )

In [ ]:
from llama_index.core import PromptTemplate

qa_prompt_tmpl_str="""
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,
respond to the user query below only if it's related to chemistry. If the query is not related to chemistry, respond with "I only answer questions related to chemistry." Do not add information from the context if it is not chemistry-related. If you don't know the answer to a chemistry question, simply respond with "I don't know." Do not attempt to fabricate an answer.
If context is empty or if there is no context present above then say "I don't have information to answer the query." Don't try to make up an answer when the query is not chemistry-related.
Query: {query_str}
Answer:
"""


# qa_prompt_tmpl_str="""
# Context information is below.
# ---------------------
# {context_str}
# ---------------------
# Given the context information and not prior knowledge,
# respond to the user query below only if it's related to chemistry. If the query is not related to chemistry, respond with "I only answer questions related to chemistry." Do not add information from the context if it is not chemistry-related. If you don't know the answer to a chemistry question, simply respond with "I don't know." Do not attempt to fabricate an answer.
# if context is empty then say I don't have information to answer the query. \
# don't try to make up an answer when query is not chemistry related. \
# Query: {query_str}
# Answer:
# """

# qa_prompt_tmpl_str = """\
# Context information is below.
# ---------------------
# {context_str}
# ---------------------
# Given the context information and not prior knowledge, \
# Given the user query below check if it's related to chemistry subject if it's not then say "I only answer questions related to chemistry" and don't try to add from the context. \
# Answer the query asking about chemistry related topics only. \
# If you don't know the answer, just say you don't know, don't try to make up an answer. \
# Query: {query_str}
# Answer: \
# """

qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str
)

In [ ]:
from typing import Any, Dict, List

from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.llama_pack.base import BaseLlamaPack
from llama_index.core.schema import Document, NodeWithScore
from llama_index.core.query_pipeline.query import QueryPipeline
from llama_index.tools.tavily_research.base import TavilyToolSpec
from llama_index.core.prompts import PromptTemplate

system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
    Retrieved document: \n\n {context_str}
    \n\n User question: {query_str}
    \n\n Evaluation('yes' or 'no'):"""
DEFAULT_RELEVANCY_PROMPT_TEMPLATE = PromptTemplate(
    template=system
)
# DEFAULT_RELEVANCY_PROMPT_TEMPLATE = PromptTemplate(
# #     template1="""As a grader, your task is to evaluate the relevance of a document retrieved in response to a user's question.
# # """
# template="""Task: Evaluate Document Relevance

#     Retrieved Document:
#     -------------------
#     {context_str}

#     User Question:
#     --------------
#     {query_str}

#     Evaluation Criteria:
#     -------------------
#     Determine if the document contains keywords or topics related to the user's question.
#     The goal is to identify and filter out clearly irrelevant documents.
#     The evaluation should not be overly stringent; the primary objective is to identify and filter out clearly irrelevant retrievals.

#     Decision:
#     ---------
#     - reply with 'yes' if the document is relevant to the question.
#     - reply with 'no' if the document is not relevant.
#     - don't provide any explanation
#     - you should only respond with 'yes' or 'no'.
#     - document relevant (yes or no): """
# )

DEFAULT_TRANSFORM_QUERY_TEMPLATE = PromptTemplate(
    template="""Your task is to refine a query to ensure it is highly effective for retrieving relevant search results. \n
    Analyze the given input to grasp the core semantic intent or meaning. \n
    Original Query:
    \n ------- \n
    {query_str}
    \n ------- \n
    Your goal is to rephrase or enhance this query to improve its search performance. Ensure the revised query is concise and directly aligned with the intended search objective. \n
    Respond with the optimized query only:"""
)
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer(response_mode="refine")

class CorrectiveRAG(BaseLlamaPack):
    def __init__(self, index, tavily_ai_apikey: str,llm) -> None:
        """Init params."""
        self.llm = llm
        self.relevancy_pipeline = QueryPipeline(
            chain=[DEFAULT_RELEVANCY_PROMPT_TEMPLATE, llm]
        )
        self.transform_query_pipeline = QueryPipeline(
            chain=[DEFAULT_TRANSFORM_QUERY_TEMPLATE, llm]
        )
        self.index = index
        self.tavily_tool = TavilyToolSpec(api_key=tavily_ai_apikey)

    def get_modules(self) -> Dict[str, Any]:
        """Get modules."""
        return {"llm": self.llm, "index": self.index}

    def retrieve_nodes(self, query_str: str, **kwargs: Any) -> List[NodeWithScore]:
        """Retrieve the relevant nodes for the query."""
        retriever = self.index.as_retriever(**kwargs)
        a = retriever.retrieve(query_str)
        return a

    def evaluate_relevancy(
        self, retrieved_nodes: List[Document], query_str: str
    ) -> List[str]:
        """Evaluate relevancy of retrieved documents with the query."""
        relevancy_results = []
        for node in retrieved_nodes:
            relevancy = self.relevancy_pipeline.run(
                context_str=node.text, query_str=query_str
            )
            relevancy_results.append(relevancy.text)
        return relevancy_results

    def extract_relevant_texts(
        self, retrieved_nodes: List[NodeWithScore], relevancy_results: List[str]
    ) -> str:
        """Extract relevant texts from retrieved documents."""
        relevant_texts = [
            retrieved_nodes[i].text
            for i, result in enumerate(relevancy_results)
            if "yes" in result or "Yes" in result or "YES" in result
        ]
        return "\n".join(relevant_texts)

    def search_with_transformed_query(self, query_str: str) -> str:
        """Search the transformed query with Tavily API."""
        search_results = self.tavily_tool.search(query_str, max_results=5)
        return "\n".join([result.text for result in search_results])

    def get_result(self, relevant_text: str, search_text: str, query_str: str) -> Any:
        """Get result with relevant text."""
        documents = [Document(text=relevant_text + "\n" + search_text)]
        if len(documents[0].text) <=2:
            return "I don't have information to answer the query."
        # index = VectorStoreIndex.from_documents(documents, optimize_for='speed')
        index = SummaryIndex.from_documents(documents)
        query_engine = index.as_query_engine(llm=llm,text_qa_template=qa_prompt_tmpl)
        return query_engine.query(query_str)
        # response_synthesizer=response_synthesizer,

    def run(self, query_str: str, **kwargs: Any) -> Any:
        """Run the pipeline."""
        # Retrieve nodes based on the input query string.
        retrieved_nodes = self.retrieve_nodes(query_str, **kwargs)

        # Evaluate the relevancy of each retrieved document in relation to the query string.
        relevancy_results = self.evaluate_relevancy(retrieved_nodes, query_str)

        # Extract texts from documents that are deemed relevant based on the evaluation.
        relevant_text = self.extract_relevant_texts(retrieved_nodes, relevancy_results)

        # Initialize search_text variable to handle cases where it might not get defined.
        search_text = ""

        # If any document is found irrelevant, transform the query string for better search results.
        for c in relevancy_results:
             if "yes" not in c.lower():
                 print("no relevant in docs")
                 transformed_query_str = self.transform_query_pipeline.run(
                     query_str=query_str
                   ).text
                 print(transformed_query_str)
            # Conduct a search with the transformed query string and collect the results.
                 search_text = self.search_with_transformed_query(transformed_query_str)
                 break


        # Compile the final result. If there's additional search text from the transformed query,
        # it's included; otherwise, only the relevant text from the initial retrieval is returned.
        if search_text:
            return self.get_result(relevant_text, search_text, query_str)
        else:
            return self.get_result(relevant_text, "", query_str)
crag = CorrectiveRAG(index,API_KEY,llm)

In [ ]:
print(crag.run("Explain Hydrogen bond"))

In [ ]:
!ngrok authtoken 2hTSXbL3QBrRB2dIPNCWko4EnBa_fibWxGZzRHSapCcbRBtA

In [ ]:
# from pyngrok import ngrok

# # [<NgrokTunnel: "https://<public_sub>.ngrok.io" -> "http://localhost:80">]
# tunnels = ngrok.get_tunnels()
# for t in tunnels:
#     print(t)
#     ngrok.disconnect(t.public_url)


In [ ]:
import os
import threading
from flask_cors import CORS
from flask import Flask
from flask import request,jsonify
from pyngrok import ngrok

app = Flask(__name__)
port = 6248
cors = CORS(app)

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

@app.route("/")
def home():
    return "Hello World!"
@app.route("/query", methods=["GET"])
def query_index():
    query_text = request.args.get("text", None)
    if query_text is None:
        return (
            "No text found, please include a ?text=blah parameter in the URL",
            400,
        )
    print(query_text)
    resp = str(crag.run(query_text))
    print(resp)
    return jsonify({"resp":resp}),200

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"port": port}).start()

In [ ]:
# # Use the public URL to make a curl request
# !curl "localhost:6247/query?text=what+is+physcial+keyboard"


what is physcial keyboard


Llama.generate: prefix-match hit

llama_print_timings:        load time =     711.56 ms
llama_print_timings:      sample time =      32.03 ms /    52 runs   (    0.62 ms per token,  1623.48 tokens per second)
llama_print_timings: prompt eval time =     976.10 ms /   909 tokens (    1.07 ms per token,   931.26 tokens per second)
llama_print_timings:        eval time =    1897.87 ms /    51 runs   (   37.21 ms per token,    26.87 tokens per second)
llama_print_timings:       total time =    2939.85 ms /   960 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     711.56 ms
llama_print_timings:      sample time =      28.41 ms /    46 runs   (    0.62 ms per token,  1618.86 tokens per second)
llama_print_timings: prompt eval time =     886.63 ms /   888 tokens (    1.00 ms per token,  1001.54 tokens per second)
llama_print_timings:        eval time =    1736.54 ms /    45 runs   (   38.59 ms per token,    25.91 tokens per second)
llama_print_timings:       to

[' no. The document does not contain any information about a physical keyboard. It appears to be the copyright and publication information for a textbook on Chemistry, Part II for Class XII published by the National Council of Educational Research and Training (NCERT).', ' no. The document does not contain any information about a physical keyboard. It appears to be an excerpt from the copyright page of a chemistry textbook published by NCERT (National Council of Educational Research and Training).']
no relevant in docs



llama_print_timings:        load time =     711.56 ms
llama_print_timings:      sample time =       5.63 ms /    10 runs   (    0.56 ms per token,  1776.83 tokens per second)
llama_print_timings: prompt eval time =     193.98 ms /   119 tokens (    1.63 ms per token,   613.48 tokens per second)
llama_print_timings:        eval time =     350.09 ms /     9 runs   (   38.90 ms per token,    25.71 tokens per second)
llama_print_timings:       total time =     555.10 ms /   128 tokens




    What is a physical keyboard?


Llama.generate: prefix-match hit

llama_print_timings:        load time =     711.56 ms
llama_print_timings:      sample time =      31.77 ms /    47 runs   (    0.68 ms per token,  1479.57 tokens per second)
llama_print_timings: prompt eval time =     936.42 ms /   958 tokens (    0.98 ms per token,  1023.05 tokens per second)
llama_print_timings:        eval time =    1715.19 ms /    46 runs   (   37.29 ms per token,    26.82 tokens per second)
llama_print_timings:       total time =    2721.66 ms /  1004 tokens
INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 20:19:24] "GET /query?text=what+is+physcial+keyboard HTTP/1.1" 200 -


I only answer questions related to chemistry.
When query is about computer hardware, such as "what is a physical keyboard", I don't have information to answer the query. Don't try to make up an answer.
127.0.0.1 - - [05/Jun/2024 20:19:24] "GET /query?text=what+is+physcial+keyboard HTTP/1.1" 200 -
{"resp":"I only answer questions related to chemistry.\nWhen query is about computer hardware, such as \"what is a physical keyboard\", I don't have information to answer the query. Don't try to make up an answer."}


In [ ]:
from flask import Flask
from flask_cors import CORS
from flask import request,stream_with_context,jsonify
app = Flask(__name__)
cors = CORS(app)

@app.route("/")
def home():
    return "Hello World!"
@app.route("/query", methods=["GET"])
def query_index():
    query_text = request.args.get("text", None)
    if query_text is None:
        return (
            "No text found, please include a ?text=blah parameter in the URL",
            400,
        )
    return jsonify({"resp":str(crag.run(query_text))}),200
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000


 * Running on http://127.0.0.1:5000


INFO:werkzeug:Press CTRL+C to quit


Press CTRL+C to quit


...

no relevant in docs



llama_print_timings:        load time =   10731.43 ms
llama_print_timings:      sample time =       6.72 ms /    12 runs   (    0.56 ms per token,  1785.45 tokens per second)
llama_print_timings: prompt eval time =     154.79 ms /   125 tokens (    1.24 ms per token,   807.52 tokens per second)
llama_print_timings:        eval time =     377.25 ms /    11 runs   (   34.30 ms per token,    29.16 tokens per second)
llama_print_timings:       total time =     544.58 ms /   136 tokens




    IUPAC name of benzene


Llama.generate: prefix-match hit

llama_print_timings:        load time =   10731.43 ms
llama_print_timings:      sample time =      38.24 ms /    67 runs   (    0.57 ms per token,  1752.28 tokens per second)
llama_print_timings: prompt eval time =    2499.37 ms /  3326 tokens (    0.75 ms per token,  1330.74 tokens per second)
llama_print_timings:        eval time =    2414.66 ms /    66 runs   (   36.59 ms per token,    27.33 tokens per second)
llama_print_timings:       total time =    5006.11 ms /  3392 tokens
INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 08:30:41] "GET /query?text=What+is+the+IUPAC+name+of+C6H6+? HTTP/1.1" 200 -


127.0.0.1 - - [13/Jun/2024 08:30:41] "GET /query?text=What+is+the+IUPAC+name+of+C6H6+? HTTP/1.1" 200 -


In [ ]:
!ngrok http 5000 #run this in terminal